# Preparação dos Dados para o Dashboard (DS1)
### Este código organiza o histórico de voos e gera os arquivos JSON que serão usados para criar os gráficos da interface final 🛫📊(DS3).

In [25]:

# ==============================================================================
# CONFIGURAÇÃO DE AMBIENTE E IMPORTAÇÃO DE PIPELINE PERSONALIZADO
# ==============================================================================

import sys
import os
import zipfile
import glob
import gdown
import pandas as pd
import json

# Garantir que o Colab encontre o arquivo do script.
sys.path.append(os.getcwd())

# Importar o script de pipeline.
try:
    import flight_delay_pipeline as sc
    print("✅ Script importado com sucesso!")
except ImportError:
    print("❌ Erro: Certifique-se de que o arquivo .py está na barra lateral do Colab.")


✅ Script importado com sucesso!


In [26]:
# ==============================================================================
# CONSOLIDAÇÃO DOS DADOS E TRATAMENTO DE ENCODING (VRA)
# ==============================================================================

# Função de carregamento.
def carregar_vra(pasta="/content/dados_vra/dados_vra", padrao="VRA_*.csv"):
    caminho_busca = os.path.join(pasta, padrao)
    arquivos = sorted(glob.glob(caminho_busca))

    dfs = [pd.read_csv(arq, sep=";", encoding="latin-1", skiprows=1, low_memory=False) for arq in arquivos]
    df_final = pd.concat(dfs, ignore_index=True)

    # Renomeação das colunas para o padrão do projeto
    # Ajustando os nomes das colunas com base na interpretação 'mangelada' dos caracteres especiais
    mapa_colunas = {
        "ICAO Empresa AÃ©rea" : "empresa_aerea",
        "ICAO AerÃ³dromo Origem" : "aerodromo_origem",
        "ICAO AerÃ³dromo Destino" : "aerodromo_destino",
        "Partida Prevista" : "partida_prevista",
        "Partida Real" : "partida_real",
        "SituaÃ§Ã£o Voo" : "situacao_voo" # Corrigido o nome da coluna
    }
    return df_final.rename(columns=mapa_colunas)

df = carregar_vra()
print(f"✅ Dataset carregado com {df.shape[0]} linhas.")

✅ Dataset carregado com 3968418 linhas.


In [27]:
# ==============================================================================
# PROCESSAMENTO ANALÍTICO: REGRAS DE NEGÓCIO E GERAÇÃO DE ENTREGÁVEIS (JSON)
# ==============================================================================

# Preparar datas com suporte a diferentes formatos (ISO8601 lida com milissegundos).
print("⏳ Convertendo colunas de data...")
df['partida_prevista'] = pd.to_datetime(df['partida_prevista'], format='ISO8601')
df['partida_real'] = pd.to_datetime(df['partida_real'], format='ISO8601')

# Calcular a diferença em minutos.
# (Real - Previsto) -> positivo significa atraso.
df['minutos_atraso'] = (df['partida_real'] - df['partida_prevista']).dt.total_seconds() / 60

# REGRA DA SEMANA 4: Definir atrasado como 1 se for >= 15 minutos.
df['atrasado'] = df['minutos_atraso'].apply(lambda x: 1 if x >= 15 else 0)

# Filtrar apenas voos que foram REALIZADOS para a análise do dashboard.
df_analise = df[df['situacao_voo'] == 'REALIZADO'].copy()

# Gerar resumos para o dashboard (JSONs).
def salvar_resumo(df, coluna, nome_arquivo):
    # Calculo da média (taxa de atraso) agrupada pela coluna desejada.
    resumo = df.groupby(coluna)['atrasado'].mean().round(4).to_dict()
    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        json.dump(resumo, f, ensure_ascii=False, indent=4)
    print(f"💾 Arquivo gerado: {nome_arquivo}")

# Executar as gerações exigidas pelo cronograma.
salvar_resumo(df_analise, 'empresa_aerea', 'atraso_empresa.json')
salvar_resumo(df_analise, 'aerodromo_origem', 'atraso_origem.json')

# Gerar sazonalidade por hora.
df_analise['hora'] = df_analise['partida_prevista'].dt.hour
salvar_resumo(df_analise, 'hora', 'sazonalidade_atrasos.json')

print(f"\n✅ Análise concluída com sucesso para {df_analise.shape[0]} voos!")


⏳ Convertendo colunas de data...
💾 Arquivo gerado: atraso_empresa.json
💾 Arquivo gerado: atraso_origem.json
💾 Arquivo gerado: sazonalidade_atrasos.json

✅ Análise concluída com sucesso para 3801626 voos!


### 🛫⏰Regra de Negócio: Consideramos como voo atrasado qualquer registro com 15 minutos ou mais de diferença entre a partida real e a prevista.


In [28]:
# ==============================================================================
# CRIAÇÃO DE MÉTRICAS
# ==============================================================================

# Calculando o atraso real em minutos (Partida Real - Partida Prevista).
df_analise['atraso_minutos'] = (df_analise['partida_real'] - df_analise['partida_prevista']).dt.total_seconds() / 60

# Definindo a regra de 15 minutos.
df_analise['atraso_15m'] = (df_analise['atraso_minutos'] >= 15).astype(int)

# Criando componentes para análise de sazonalidade.
df_analise['hora_dia'] = df_analise['partida_prevista'].dt.hour
df_analise['dia_semana'] = df_analise['partida_prevista'].dt.day_name()

print("✅ Métricas 'atraso_15m' e sazonalidade criadas com sucesso.")


✅ Métricas 'atraso_15m' e sazonalidade criadas com sucesso.


In [29]:
# ==============================================================================
# GERAÇÃO DE JSON E VALIDAÇÃO VISUAL
# ==============================================================================

def exportar_e_validar(coluna, nome_arquivo):
    resumo = df_analise.groupby(coluna)['atraso_15m'].mean().reset_index()
    resumo.to_json(nome_arquivo, orient='records')
    print(f"\n📊 Validação: {coluna}")
    print(resumo.sort_values(by='atraso_15m', ascending=False).head(5))
    return resumo

# Gerando e validando os 5 entregáveis obrigatórios.
print("🚀 Iniciando exportação dos arquivos JSON...")
resumo_empresa = exportar_e_validar('empresa_aerea', 'atraso_empresa.json')
resumo_origem  = exportar_e_validar('aerodromo_origem', 'atraso_origem.json')
resumo_destino = exportar_e_validar('aerodromo_destino', 'atraso_destino.json')
resumo_hora    = exportar_e_validar('hora_dia', 'sazonalidade_atrasos.json')
resumo_semana  = exportar_e_validar('dia_semana', 'sazonalidade_dia_semana.json')

print("\n🏁 Checklist: Todos os arquivos foram salvos e os números variam (Não são uniformes).")


🚀 Iniciando exportação dos arquivos JSON...

📊 Validação: empresa_aerea
    empresa_aerea  atraso_15m
0             1DE         1.0
20            ARN         1.0
19            ARL         1.0
41            CFG         1.0
108           MSR         1.0

📊 Validação: aerodromo_origem
    aerodromo_origem  atraso_15m
490             SVBM         1.0
492             SVMG         1.0
494             SVMT         1.0
525             TAPA         1.0
19              EBOS         1.0

📊 Validação: aerodromo_destino
    aerodromo_destino  atraso_15m
0                BIKF         1.0
563              ZBTJ         1.0
29               EGPK         1.0
76               KACY         1.0
503              SVMC         1.0

📊 Validação: hora_dia
    hora_dia  atraso_15m
0        0.0    0.241690
1        1.0    0.239045
18      18.0    0.213268
17      17.0    0.212245
22      22.0    0.209365

📊 Validação: dia_semana
  dia_semana  atraso_15m
0     Friday    0.193514
4   Thursday    0.190332
6  Wednesd

In [30]:
# ==============================================================================
# FINALIZAÇÃO, CHECKLIST DE INTEGRIDADE E EXPORTAÇÃO (DELIVERY)
# ==============================================================================

# Gerar as agregações que faltavam (Aeroporto de Destino e Dia da Semana).
salvar_resumo(df_analise, 'aerodromo_destino', 'atraso_destino.json')

# Sazonalidade completa (Dia da Semana).
# Criar coluna do dia da semana.
df_analise['dia_semana'] = df_analise['partida_prevista'].dt.day_name()
salvar_resumo(df_analise, 'dia_semana', 'sazonalidade_dia_semana.json')

# Criar uma lista de conferência para garantir os entregáveis.
entregaveis = [
    'atraso_empresa.json',
    'atraso_origem.json',
    'atraso_destino.json',
    'sazonalidade_atrasos.json',
    'sazonalidade_dia_semana.json'
]

print("\n--- 🏁 CHECKLIST DE ENTREGA ---")
for arq in entregaveis:
    if os.path.exists(arq):
        print(f"✅ {arq} pronto para download.")
    else:
        print(f"❌ {arq} não encontrado. Verifique a geração.")

# Comando para baixar todos aqruivos de uma vez (Colab)
from google.colab import files
for arq in entregaveis:
    if os.path.exists(arq):
        files.download(arq)


💾 Arquivo gerado: atraso_destino.json
💾 Arquivo gerado: sazonalidade_dia_semana.json

--- 🏁 CHECKLIST DE ENTREGA ---
✅ atraso_empresa.json pronto para download.
✅ atraso_origem.json pronto para download.
✅ atraso_destino.json pronto para download.
✅ sazonalidade_atrasos.json pronto para download.
✅ sazonalidade_dia_semana.json pronto para download.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##📝 Resumo Executivo: Metodologia de Dados (DS1)

1. Regra de Negócio (Conceito de Atraso)
Para garantir a padronização das análises em todos os gráficos do dashboard, foi adotada a seguinte regra:

Definição: Um voo é considerado "atrasado" quando a diferença entre o horário de Partida Real e o horário de Partida Prevista é igual ou superior a 15 minutos.🛫⏰
​

Justificativa: Este critério segue padrões internacionais de pontualidade na aviação comercial, permitindo identificar gargalos operacionais significativos sem considerar pequenas variações irrelevantes de poucos minutos.
​

2. Período dos Dados Analisados

Fonte: Dataset Histórico de Voos (VRA - ANAC).

Janela Temporal: O dataset processado compreende o período de 01 de janeiro de 2020 até 01 de julho de 2025.🗓️
​

Volume de Dados: Foram processados 3.801.626 voos, garantindo alta relevância estatística para as médias apresentadas nos arquivos JSON.
​

